In [1]:
import pandas as pd
import numpy as np
import json
import glob

# Cleaning Points of Interest data

#### Read in uncleaned Points of Interest data and merge into a single pandas dataframe:

In [2]:
#this code assumes you have the raw data in a folder titled 'poi_data', which should be in the same folder as this file
all_node_files = glob.glob('poi_data/' + '/*node.json')

li = []

for filename in all_node_files:
    data = json.load(open(filename))
    df = pd.DataFrame(data["elements"])
    li.append(df)

all_nodes = pd.concat(li, axis=0, ignore_index=True)

#### Exploring the data with basic cleaning methods (first pass):

In [4]:
all_nodes.dropna()

,type,id,lat,lon,tags
0,node,78252154,52.523744,13.398627,"{'addr:city': 'Berlin', 'addr:country': 'DE', ..."
1,node,87036263,52.532985,13.384282,"{'amenity': 'atm', 'brand': 'Sparda-Bank', 'br..."
2,node,89275133,52.518025,13.406956,"{'amenity': 'atm', 'name': 'Bankhaus August Le..."
3,node,213106623,52.542170,13.441137,"{'addr:country': 'DE', 'addr:housenumber': '87..."
4,node,213113204,52.542750,13.392862,"{'amenity': 'atm', 'name': 'Berliner Sparkasse..."
...,...,...,...,...,...
213006,node,8931299152,52.487989,13.275393,{'tourism': 'viewpoint'}
213007,node,9024702237,52.506772,13.334563,{'tourism': 'viewpoint'}
213008,node,9026936271,52.401704,13.366960,"{'name': 'Alpengipfel', 'tourism': 'viewpoint'}"
213009,node,9038673666,52.482133,13.291911,"{'direction': 'E', 'tourism': 'viewpoint'}"


There are no missing values

## Extracting relevant features from 'tags' attribute

#### Extracting the name of each POI and adding it as a separate feature 

In [5]:
# extracting the 'name' data and adding it to a new 'name' column 
for i, dict in enumerate(all_nodes.tags):
    try:
        all_nodes.loc[i, 'name'] = dict['name']
    except:
        pass

all_nodes

,type,id,lat,lon,tags,name
0,node,78252154,52.523744,13.398627,"{'addr:city': 'Berlin', 'addr:country': 'DE', ...",Bank für Sozialwirtschaft
1,node,87036263,52.532985,13.384282,"{'amenity': 'atm', 'brand': 'Sparda-Bank', 'br...",Sparda-Bank
2,node,89275133,52.518025,13.406956,"{'amenity': 'atm', 'name': 'Bankhaus August Le...",Bankhaus August Lenz
3,node,213106623,52.542170,13.441137,"{'addr:country': 'DE', 'addr:housenumber': '87...",NaN
4,node,213113204,52.542750,13.392862,"{'amenity': 'atm', 'name': 'Berliner Sparkasse...",Berliner Sparkasse
...,...,...,...,...,...,...
213006,node,8931299152,52.487989,13.275393,{'tourism': 'viewpoint'},NaN
213007,node,9024702237,52.506772,13.334563,{'tourism': 'viewpoint'},NaN
213008,node,9026936271,52.401704,13.366960,"{'name': 'Alpengipfel', 'tourism': 'viewpoint'}",Alpengipfel
213009,node,9038673666,52.482133,13.291911,"{'direction': 'E', 'tourism': 'viewpoint'}",NaN


### Extracting relevant categories

In [7]:
# dictionary of category key/value pairs that were scraped
categories = {'amenity': ['atm',
                          'bar',
                          'bench',
                          'cafe',
                          'drinking_water',
                          'ice_cream',
                          'nightclub',
                          'restaurant'
                         ],
              
              'historic': 'yes',
              'leisure': 'picnic_table',
              'natural': 'tree',
              'shop':'convenience',
              'tourism': ['art_work',
                          'attraction',
                          'gallery',
                          'museum',
                          'viewpoint',
                          ]
              }


In [ ]:
# loop through every row in all_nodes.tags, keeping track of the index
# within each row, loop through every key from categories variable
# if the row contains a matching key
# check the key value from the row against the key values from categories variable
# if these check out
# for the current row, add the value to a new column 'category'

In [32]:
for index, row in enumerate(all_nodes.tags):
        for key in categories:
            if key in row:
                for value in categories[key]:
                    if value in row[key]:
                        all_nodes.loc[index, 'category'] = row[key]            

## Inspecting the data

In [22]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 1000)

In [33]:
all_nodes.category

0               atm
1               atm
2               atm
3               atm
4               atm
            ...    
213006    viewpoint
213007    viewpoint
213008    viewpoint
213009    viewpoint
213010    viewpoint
Name: category, Length: 213011, dtype: object

In [38]:
all_nodes.loc[150000]

type                                                     node
id                                                 5322155730
lat                                                 52.475372
lon                                                 13.289718
tags        {'leaf_cycle': 'deciduous', 'leaf_type': 'broa...
name                                                      NaN
category                                                 tree
Name: 150000, dtype: object

In [39]:
all_nodes.to_json('all_nodes_with_categories.json')

In [40]:
all_nodes.to_csv('all_nodes_with_categories.csv')